In [1]:
import sys, os

In [2]:
import sys, os
import numpy as np
import pandas as pd
import pickle

In [3]:
sys.path.append(os.path.abspath('../../'))
from utils import utils
from utils.utils import evaluate_experiment
sys.path.append(os.path.abspath('../'))
from timeseries_utils import *

In [25]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from scipy.fftpack import fft
from torch.autograd import Variable
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler

## Printing configs

In [5]:
print('torch      :', torch.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
iscuda = torch.cuda.is_available()
if iscuda:
    print(f'device     : {torch.cuda.get_device_name(0)}')

torch      : 1.4.0
numpy      : 1.18.1
pandas     : 1.0.4
device     : GeForce GTX 1080


## Loading data

In [6]:
path_to_data='../../../'

In [7]:
data=np.load(os.path.abspath(path_to_data+'data-002.npy'),allow_pickle=True)
Y=np.load(path_to_data+'Y.npy',allow_pickle=True)
labels =pd.read_csv(path_to_data+'labels.csv')

In [8]:
train_fold=8
val_fold=9
test_fold=10

# 10th fold for testing (9th for now)
X_test = data[labels.strat_fold == test_fold]
y_test = Y[labels.strat_fold == test_fold]
# 9th fold for validation (8th for now)
X_val = data[labels.strat_fold == val_fold]
y_val = Y[labels.strat_fold == val_fold]
# rest for training
X_train = data[labels.strat_fold <= train_fold]
y_train = Y[labels.strat_fold <= train_fold]

In [9]:
X_train.shape ,X_val.shape

((17111, 1000, 12), (2156, 1000, 12))

## Preprocess signal data

In [10]:
X_train, X_val, X_test = utils.preprocess_signals(X_train, X_val, X_test,'/content/')
n_classes = y_train.shape[1]
X_train = np.reshape(X_train,[X_train.shape[0],X_train.shape[2],X_train.shape[1]])
X_val = np.reshape(X_val,[X_val.shape[0],X_val.shape[2],X_val.shape[1]])
X_test = np.reshape(X_test,[X_test.shape[0],X_test.shape[2],X_test.shape[1]])

In [11]:
X_train.shape, X_val.shape

((17111, 12, 1000), (2156, 12, 1000))

In [12]:
y_train.shape, y_val.shape

((17111, 5), (2156, 5))

## Get fourier transform

In [13]:
N=1000
X_train_fft = fft(X_train)
X_train_fft = (2/N) * np.abs(X_train_fft)

X_val_fft = fft(X_val)
X_val_fft = (2/N) * np.abs(X_val_fft)

In [15]:
X_train, X_val = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train, X_val)]
X_train_fft, X_val_fft = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train_fft, X_val_fft)]
y_train, y_val = [torch.tensor(arr, dtype=torch.long) for arr in (y_train, y_val)]
train_ds = TensorDataset(X_train,X_train_fft, y_train)
valid_ds = TensorDataset(X_val,X_val_fft, y_val)

## Defining blocks

In [16]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.
    
    The separable convlution is a method to reduce number of the parameters 
    in the deep learning network for slight decrease in predictions quality.
    """
    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.
    
    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """
    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):
    
        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x): 
        return self.layers(x)


class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [26]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

## Creating the Branched Model

In [22]:
class Classifier(nn.Module):
    def __init__(self, raw_ni, fft_ni, no, drop=.5):
        super().__init__()
        
        self.raw = nn.Sequential(
            SepConv1d(raw_ni,  32, 5, 2, 3, drop=drop),
            SepConv1d(    32,  64, 5, 4, 2, drop=drop),
            SepConv1d(    64, 128, 5, 4, 2, drop=drop),
            SepConv1d(   128, 256, 5, 4, 2),
            Flatten(),
            nn.Dropout(drop), nn.Linear(256, 64), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 64, 64), nn.ReLU(inplace=True))
        
        self.fft = nn.Sequential(
            SepConv1d(fft_ni,  32, 5, 2, 3, drop=drop),
            SepConv1d(    32,  64, 5, 4, 2, drop=drop),
            SepConv1d(    64, 128, 5, 4, 2, drop=drop),
            SepConv1d(   128, 256, 5, 4, 2),
            Flatten(),
            nn.Dropout(drop), nn.Linear(256, 64), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 64, 64), nn.ReLU(inplace=True))
        
        self.out = nn.Sequential(
            nn.Linear(128, 64), nn.ReLU(inplace=True), nn.Linear(64, no))
        
    def forward(self, t_raw, t_fft):
        raw_out = self.raw(t_raw)
        fft_out = self.fft(t_fft)
        t_in = torch.cat([raw_out, fft_out], dim=1)
        out = self.out(t_in)
        return out

In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#batch size
bs=256
jobs=1
train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=jobs)
valid_dl = DataLoader(valid_ds, bs, shuffle=False, num_workers=jobs)

In [27]:

lr = 0.01
#lr = 0.0005
n_epochs = 100
iterations_per_epoch = len(train_ds)
num_classes = 5
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Classifier(X_train.shape[1], X_train_fft.shape[1], num_classes).to(device)
#model = fcn_wang(num_classes=5,input_channels=X_train.shape[1]).to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

#criterion = nn.CrossEntropyLoss()
#opt = torch.optim.RMSprop(model.parameters(), lr=lr)
#sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

In [29]:

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(train_dl):
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        #x_raw, y_batch = [t.to(device) for t in batch]
        
        opt.zero_grad()

        out = model(x_raw, x_fft)
        #out = model(x_raw)
        
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        #sched.step()
        
    epoch_loss /= len(train_ds)
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    for batch in valid_dl:
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        out = model(x_raw, x_fft)
        
        #x_raw, y_batch = [t.to(device) for t in batch]
        #out = model(x_raw)
        
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    #if epoch % base == 0:
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
    #base *= step

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training


RuntimeError: size mismatch, m1: [256 x 2048], m2: [256 x 64] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:290

In [ ]:
preds = []
print('Predicting on test dataset')
for batch in valid_dl:
    x_raw, y_batch = [t.to(device) for t in batch]
    #batch = batch.permute(0, 2, 1)
    out = model(x_raw)
    y_hat = F.log_softmax(out, dim=1)
    preds += y_hat.tolist()
preds = np.asarray(preds)

In [ ]:
preds.shape

In [ ]:
from utils.utils import evaluate_experiment

thresholds=None
tr_df_point = evaluate_experiment( y_val, preds, thresholds)
print(tr_df_point)

# To test: add a branch with wavelet

# To test: Search for something like mel